# Partner-Facing Reports with Tinman

This notebook demonstrates how to generate comprehensive reports
for different stakeholders:
- Executive summaries for leadership
- Technical reports for engineering
- Compliance reports for audit/governance

In [ ]:
from datetime import datetime, timedelta

from tinman import create_tinman
from tinman.config.modes import OperatingMode
from tinman.reporting import (
    ExecutiveSummaryReport,
    TechnicalAnalysisReport,
    ComplianceReport,
    ReportFormat,
    export_report,
    export_all_formats,
)
from tinman.core.cost_tracker import CostTracker, BudgetPeriod

## Setup

Create a Tinman instance and run some research to generate data.

In [ ]:
async def setup():
    # Create Tinman with cost tracking
    tinman = await create_tinman(
        mode=OperatingMode.LAB,
        skip_db=True,
    )
    
    # Create cost tracker
    cost_tracker = CostTracker(
        budget_usd=50.0,
        period=BudgetPeriod.DAILY,
    )
    
    # Run research to generate findings
    print("Running research cycles to generate data...")
    
    for focus in ["tool_use", "instruction_following", "context_handling"]:
        await tinman.research_cycle(
            focus=focus,
            max_hypotheses=2,
            max_experiments=1,
            runs_per_experiment=3,
        )
        
        # Simulate costs
        cost_tracker.record_cost(
            amount_usd=0.50,
            source="research_cycle",
            model="claude-3",
            operation=focus,
        )
    
    state = tinman.get_state()
    print(f"\nGenerated:")
    print(f"  Failures discovered: {state['failures_discovered']}")
    print(f"  Hypotheses tested: {state['hypotheses_generated']}")
    print(f"  Experiments run: {state['experiments_run']}")
    
    return tinman, cost_tracker

tinman, cost_tracker = await setup()

## 1. Executive Summary Report

High-level report suitable for leadership and stakeholders.

In [ ]:
# Create executive report generator
exec_generator = ExecutiveSummaryReport(
    graph=tinman.graph,
    cost_tracker=cost_tracker,
)

# Generate report for last 7 days
now = datetime.utcnow()
week_ago = now - timedelta(days=7)

exec_report = await exec_generator.generate(
    period_start=week_ago,
    period_end=now,
)

print("Executive Summary Report")
print("=" * 50)
print(f"\nSummary:\n{exec_report.summary}")
print(f"\nReport contains {len(exec_report.sections)} sections")

In [ ]:
# Export to markdown
markdown = exec_generator.format(exec_report, ReportFormat.MARKDOWN)
print(markdown[:2000])  # First 2000 chars

In [ ]:
# Export to HTML (viewable in browser)
html = exec_generator.format(exec_report, ReportFormat.HTML)

# Save to file
from pathlib import Path
output_path = Path("./reports")
output_path.mkdir(exist_ok=True)

with open(output_path / "executive_summary.html", "w") as f:
    f.write(html)

print(f"HTML report saved to {output_path / 'executive_summary.html'}")

## 2. Technical Analysis Report

Detailed technical report for engineering teams.

In [ ]:
# Create technical report generator
tech_generator = TechnicalAnalysisReport(graph=tinman.graph)

# Generate report with filters
tech_report = await tech_generator.generate(
    period_start=week_ago,
    period_end=now,
    include_resolved=True,
    severity_filter=["S2", "S3", "S4"],  # Focus on medium+ severity
)

print("Technical Analysis Report")
print("=" * 50)
print(f"\nSummary:\n{tech_report.summary}")

# Show data breakdown
data = tech_report.raw_data
print(f"\nFailure breakdown:")
for cls, count in data.failure_by_class.items():
    print(f"  {cls}: {count}")

print(f"\nCommon triggers:")
for trigger, count in data.common_triggers[:5]:
    print(f"  {trigger}: {count}x")

In [ ]:
# Export technical report to markdown
tech_markdown = tech_generator.format(tech_report, ReportFormat.MARKDOWN)

with open(output_path / "technical_analysis.md", "w") as f:
    f.write(tech_markdown)

print(f"Technical report saved to {output_path / 'technical_analysis.md'}")
print(f"\nFirst 1500 characters:\n")
print(tech_markdown[:1500])

## 3. Compliance Report

Audit-ready report for governance and regulatory purposes.

In [ ]:
# For compliance reports, we need the audit logger
from tinman.db.audit import AuditLogger

# Create mock audit logger for demo
# In production, this would connect to your database
audit_logger = None  # Would be: AuditLogger(db_session)

# Create compliance report generator
compliance_generator = ComplianceReport(
    audit_logger=audit_logger,
    graph=tinman.graph,
)

# Generate report
compliance_report = await compliance_generator.generate(
    period_start=now - timedelta(days=30),  # Last 30 days
    period_end=now,
)

print("Compliance Report")
print("=" * 50)
print(f"\nSummary:\n{compliance_report.summary}")
print(f"\nSections:")
for section in compliance_report.sections:
    print(f"  - {section.title}")

## 4. Batch Export to Multiple Formats

In [ ]:
# Export all reports to multiple formats
from tinman.reporting import export_all_formats

# Export executive report to all formats
results = export_all_formats(
    exec_report,
    output_dir=output_path,
    base_name="executive_summary_full",
    formats=[ReportFormat.JSON, ReportFormat.MARKDOWN, ReportFormat.HTML],
)

print("Exported files:")
for fmt, path in results.items():
    print(f"  {fmt.value}: {path}")

## 5. Customizing Reports

In [ ]:
# Access raw JSON data for custom processing
import json

json_str = exec_generator.format(exec_report, ReportFormat.JSON)
report_data = json.loads(json_str)

print("Report structure:")
print(json.dumps({
    "metadata": report_data["metadata"],
    "section_titles": [s["title"] for s in report_data["sections"]],
}, indent=2))

In [ ]:
# Create a custom summary from report data
def create_slack_summary(report_data: dict) -> str:
    """Create a Slack-friendly summary."""
    lines = [
        f":clipboard: *{report_data['metadata']['title']}*",
        f"_{report_data['metadata']['type']}_",
        "",
        report_data['summary'][:500],
        "",
        f":calendar: Period: {report_data['metadata']['period_start'][:10]} to {report_data['metadata']['period_end'][:10]}",
    ]
    return "\n".join(lines)

slack_msg = create_slack_summary(report_data)
print("Slack message:")
print(slack_msg)

## 6. Scheduling Reports

Example of setting up automated report generation.

In [ ]:
async def generate_daily_reports(tinman, output_dir: Path):
    """Generate all daily reports."""
    now = datetime.utcnow()
    yesterday = now - timedelta(days=1)
    
    reports = []
    
    # Executive summary
    exec_gen = ExecutiveSummaryReport(graph=tinman.graph)
    exec_report = await exec_gen.generate(
        period_start=yesterday,
        period_end=now,
    )
    reports.append(("executive", exec_report, exec_gen))
    
    # Technical report
    tech_gen = TechnicalAnalysisReport(graph=tinman.graph)
    tech_report = await tech_gen.generate(
        period_start=yesterday,
        period_end=now,
    )
    reports.append(("technical", tech_report, tech_gen))
    
    # Export all
    date_str = now.strftime("%Y%m%d")
    for name, report, gen in reports:
        base_name = f"{name}_{date_str}"
        export_all_formats(
            report,
            output_dir=output_dir,
            base_name=base_name,
        )
        print(f"Generated {name} report: {base_name}")
    
    return reports

# Run daily report generation
reports = await generate_daily_reports(tinman, output_path)
print(f"\nGenerated {len(reports)} daily reports")

## Cleanup

In [ ]:
await tinman.close()
print("Session closed.")

## Next Steps

- Set up automated report delivery via email/Slack
- Create custom report templates
- Integrate with your BI tools (Tableau, Looker, etc.)
- Build dashboards using the JSON export format